In [1]:
#create file
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [13]:
!pip install transformers

In [14]:
#GetOldTweets3 --querysearch "corona virus" --lang en --maxtweets

# df[6] = tweets
# df[7] = labels
df = pd.read_csv("output.csv", delimiter=',', header=None)
df.head()


,0,1
0,text,sentiment
1,They closing Florida’s bars again. #coronavirus,negative
2,#UK Government has been stockpiling life-savin...,negative
3,Widely Available Drug Improves Survival From C...,neutral
4,Coronavirus Lockdowns Might Make Mao Zedong Pr...,neutral


In [15]:
# Load the pretrain model
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

##Want BERT instead of distilBERT? Uncomment the following line:
# model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer,
# 'bert-base-uncased')

#pr Load pre-trained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)


In [16]:

# TOKENIZING
tokenized = df[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))


In [17]:
# Padding the tokenizing list so they're all the same size
# so it can be processed by BERT all at once
max_len =0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
np.array(padded).shape

(1500, 225)

In [6]:
# MASKING to ignore the padding we added when processing the input
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(1500, 225)

In [ ]:
# Run the input through the model
input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
# slice the first token of the output which is added by BERT. It is a classification
# token that is added at the begining of every sentence

features = last_hidden_states[0][:,0,:].numpy()
labels = df.iloc[:,7] #indicates positive or negative



In [9]:
# MODEL 2 :  TRAIN TEST and SPLIT
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

# Logistic regression
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [10]:
# Evaluate the model
print("The accuracy of our model: ",  lr_clf.score(test_features, test_labels))

The accuracy of our model:  0.4026666666666667
